# BKM:
*****
>1. Config slaves as "cluster palcement group", to get best and stable throughput among nodes.
2. Open port accesses internally, don't expose port access to public
3. Configure security.authorization and ip control through hadoop
4. Network throughput depends on instance type. If master node is small instance, avoid to copy large file from master to slaves or HDFS, using one of slavers
5. If you want to cache the files to memory, set replication to 1. Otherwise you can't make sure Yarn schedule the same task to the same node always.
6. If you copy large file from one slave to whole HDFS, the distribution is biased. The slave holds most of the data in its HDFS. Solution is to copy the file from HDFS to HDFS again
7. vcpu isn't map to physical CPU with the same index, so you can't make sure two vcpu doesn't share the same physical core. So pin executors to cores through the native OS policy leads to poor performance. We need to make clear how vcpu share the same core firstly
8. 
*****

# Initialize

In [ ]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
clients=!cat /etc/hosts
clients=[l.split(" ")[0] for l in clients if l.startswith("172")]
masterip='172.31.31.159'
hclients=clients.copy()
hclients.append(masterip)

In [ ]:
user="sparkuser"

In [ ]:
%cd ~

In [ ]:
!ssh-keygen

In [ ]:
!pip install findspark

In [ ]:
!pip install pandas numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install pandasql

# create user

In [ ]:
for l in clients:
    !ssh root@$l adduser {user}

In [ ]:
for i in clients:
    !ssh root@$i cp -r .ssh /home/{user}/
    !ssh root@$i chown -R {user}:{user} /home/{user}/.ssh

# install binary

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

In [ ]:
!tar -zxvf spark-3.0.0-bin-hadoop2.7.tgz >/dev/null 2>&1

In [ ]:
!wget https://archive.apache.org/dist/hadoop/core/hadoop-2.7.0/hadoop-2.7.0.tar.gz > /dev/null 2>&1

In [ ]:
!tar zxvf hadoop-2.7.0.tar.gz > /dev/null 2>&1

In [ ]:
#!wget https://download.java.net/openjdk/jdk8u40/ri/openjdk-8u40-b25-linux-x64-10_feb_2015.tar.gz > /dev/null 2>&1

In [ ]:
!tar zxvf jdk1.8.0_212.tgz > /dev/null 2>&1

## same performance using openjdk and jdk

# config clients

In [ ]:
for l in hclients:
    !ssh root@$l "mkdir -p /data/1/{user}; chown {user}:{user} /data/1/{user}"

In [ ]:
for l in hclients:
    !ssh root@$l "mkdir -p /data/1/tmp; chown {user}:{user} /data/1/tmp"

In [ ]:
for l in hclients:
    !ssh $l "mkdir -p /data/1/{user}/hdfs/data; mkdir -p /data/1/{user}/yarn; "

In [ ]:
for l in hclients:
    !ssh $l "mkdir -p /data/1/{user}/hdfs/name; mkdir -p /data/1/{user}/name; "

In [ ]:
for l in clients:
    !scp hadoop-2.7.0.tar.gz $l:~/

In [ ]:
for l in clients:
    !scp jdk1.8.0_212.tgz $l:~/

In [ ]:
for l in clients:
    !ssh $l "tar zxvf hadoop-2.7.0.tar.gz > /dev/null 2>&1"

In [ ]:
for l in clients:
    !ssh $l "tar zxvf jdk1.8.0_212.tgz > /dev/null 2>&1"

In [ ]:
for l in hclients:
    !ssh $l "mkdir spark "

In [ ]:
cfg='''export HADOOP_HOME=~/hadoop-2.7.0
export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
export YARN_CONF_DIR=$HADOOP_HOME/etc/hadoop
export SPARK_LOCAL_DIRS=~/spark

export JAVA_HOME=~/jdk1.8.0_212
export PATH=$JAVA_HOME/bin:$PATH
export SPARK_HOME=~/spark-3.0.0-bin-hadoop2.7
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.4-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$PATH

'''

In [ ]:
with open("tmpcfg",'w') as f:
    f.writelines(cfg)

In [ ]:
for l in hclients:
    !scp tmpcfg $l:~/tmpcfg.in
    !ssh $l "cat ~/tmpcfg.in >> ~/.bashrc"

# config Hadoop

## enable security.authorization

In [ ]:
coresite='''<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
      <name>fs.default.name</name>
      <value>hdfs://{:s}:8020</value>
      <final>true</final>
  </property>
  <property>
      <name>hadoop.security.authentication</name>
      <value>simple</value>
  </property>
  <property>
      <name>hadoop.security.authorization</name>
      <value>true</value>
  </property>
</configuration>
'''.format(masterip)

with open('hadoop-2.7.0/etc/hadoop/core-site.xml','w') as f:
    f.writelines(coresite)
    
for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/core-site.xml $l:~/hadoop-2.7.0/etc/hadoop/core-site.xml >/dev/null 2>&1

## set IP check, note the command <font color=red>", "</font>.join

In [ ]:
hadooppolicy='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--

 Licensed to the Apache Software Foundation (ASF) under one
 or more contributor license agreements.  See the NOTICE file
 distributed with this work for additional information
 regarding copyright ownership.  The ASF licenses this file
 to you under the Apache License, Version 2.0 (the
 "License"); you may not use this file except in compliance
 with the License.  You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>security.service.authorization.default.hosts</name>
    <value>{:s}</value>
  </property>
  <property>
    <name>security.service.authorization.default.acl</name>
    <value>{:s} {:s}</value>
  </property>
  
  
    <property>
    <name>security.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientProtocol, which is used by user code
    via the DistributedFileSystem.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.client.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientDatanodeProtocol, the client-to-datanode protocol
    for block recovery.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for DatanodeProtocol, which is used by datanodes to
    communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.inter.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for InterDatanodeProtocol, the inter-datanode protocol
    for updating generation timestamp.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.namenode.protocol.acl</name>
    <value>*</value>
    <description>ACL for NamenodeProtocol, the protocol used by the secondary
    namenode to communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

 <property>
    <name>security.admin.operations.protocol.acl</name>
    <value>*</value>
    <description>ACL for AdminOperationsProtocol. Used for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.user.mappings.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshUserMappingsProtocol. Used to refresh
    users mappings. The ACL is a comma-separated list of user and
    group names. The user and group list is separated by a blank. For
    e.g. "alice,bob users,wheel".  A special value of "*" means all
    users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.policy.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshAuthorizationPolicyProtocol, used by the
    dfsadmin and mradmin commands to refresh the security policy in-effect.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.ha.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for HAService protocol used by HAAdmin to manage the
      active and stand-by states of namenode.</description>
  </property>

  <property>
    <name>security.zkfc.protocol.acl</name>
    <value>*</value>
    <description>ACL for access to the ZK Failover Controller
    </description>
  </property>

  <property>
    <name>security.qjournal.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for QJournalProtocol, used by the NN to communicate with
    JNs when using the QuorumJournalManager for edit logs.</description>
  </property>

  <property>
    <name>security.mrhs.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for HSClientProtocol, used by job clients to
    communciate with the MR History Server job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <!-- YARN Protocols -->

  <property>
    <name>security.resourcetracker.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceTrackerProtocol, used by the
    ResourceManager and NodeManager to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcemanager-administration.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceManagerAdministrationProtocol, for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationclient.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationClientProtocol, used by the ResourceManager
    and applications submission clients to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationmaster.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationMasterProtocol, used by the ResourceManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.containermanagement.protocol.acl</name>
    <value>*</value>
    <description>ACL for ContainerManagementProtocol protocol, used by the NodeManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcelocalizer.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceLocalizer protocol, used by the NodeManager
    and ResourceLocalizer to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.task.protocol.acl</name>
    <value>*</value>
    <description>ACL for TaskUmbilicalProtocol, used by the map and reduce
    tasks to communicate with the parent tasktracker.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for MRClientProtocol, used by job clients to
    communciate with the MR ApplicationMaster to query job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationhistory.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationHistoryProtocol, used by the timeline
    server and the generic history service client to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  
  
  
  
</configuration>
'''.format((", ").join(hclients),user,user)

with open('hadoop-2.7.0/etc/hadoop/hadoop-policy.xml','w') as f:
    f.writelines(hadooppolicy)
    
for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/hadoop-policy.xml $l:~/hadoop-2.7.0/etc/hadoop/hadoop-policy.xml >/dev/null 2>&1


In [ ]:
x=4
f"{:s}x{:d}".format("ss",x)

## hdfs config, set replication to 1 to cache all the data

In [ ]:
hdfs_site=f'''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>dfs.namenode.secondary.http-address</name>
    <value>{masterip}:50090</value>
  </property>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>/data/1/{user}/hdfs/name</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.datanode.data.dir</name>
        <value>/data/1/{user}/hdfs/data</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.namenode.checkpoint.dir</name>
    <value>/data/1/{user}/hdfs/namesecondary</value>
    <final>true</final>
  </property>
  <property>
    <name>dfs.name.handler.count</name>
    <value>100</value>
  </property>
  <property>
    <name>dfs.blocksize</name>
    <value>128m</value>
</property>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
</property>
</configuration>
'''


with open('hadoop-2.7.0/etc/hadoop/hdfs-site.xml','w') as f:
    f.writelines(hdfs_site)
    
for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/hdfs-site.xml $l:~/hadoop-2.7.0/etc/hadoop/hdfs-site.xml >/dev/null 2>&1


## mapreduce config

In [ ]:
mapreduce='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>

     <property>
         <name>mapreduce.job.maps</name>
         <value>288</value>
     </property>
     <property>
         <name>mapreduce.job.reduces</name>
         <value>64</value>
     </property>

     <property>
         <name>mapreduce.map.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
      <property>
         <name>mapreduce.map.memory.mb</name>
         <value>6144</value>
         </property>

     <property>
         <name>mapreduce.reduce.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
     <property>
         <name>mapreduce.reduce.memory.mb</name>
         <value>6144</value>
     </property>
     <property>
         <name>yarn.app.mapreduce.am.staging-dir</name>
         <value>/user</value>
     </property>
     <property>
         <name>mapreduce.task.io.sort.mb</name>
         <value>2000</value>
     </property>
     <property>
         <name>mapreduce.task.timeout</name>
         <value>3600000</value>
     </property>
<!-- MapReduce Job History Server security configs -->
<property>
  <name>mapreduce.jobhistory.address</name>
  <value>{:s}:10020</value>
</property>

</configuration>
'''.format(masterip)


with open('hadoop-2.7.0/etc/hadoop/mapred-site.xml','w') as f:
    f.writelines(mapreduce)
    
for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/mapred-site.xml $l:~/hadoop-2.7.0/etc/hadoop/mapred-site.xml >/dev/null 2>&1


## yarn config

In [ ]:
yarn_site=f'''<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
  <property>
      <name>yarn.resourcemanager.hostname</name>
      <value>{masterip}</value>
  </property>
  <property>
      <name>yarn.resourcemanager.address</name>
      <value>{masterip}:8032</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.memory-mb</name>
      <value>190000</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.cpu-vcores</name>
      <value>96</value>
  </property>
  <property>
      <name>yarn.nodemanager.pmem-check-enabled</name>
      <value>false</value>
  </property>

  <property>
      <name>yarn.nodemanager.vmem-check-enabled</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.vmem-pmem-ratio</name>
      <value>4.1</value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services</name>
      <value>mapreduce_shuffle,spark_shuffle</value>
  </property>

  <property>
      <name>yarn.scheduler.minimum-allocation-mb</name>
      <value>1024</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-mb</name>
      <value>190000</value>
  </property>
  <property>
      <name>yarn.scheduler.minimum-allocation-vcores</name>
      <value>1</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-vcores</name>
      <value>96</value>
  </property>

  <property>
      <name>yarn.log-aggregation-enable</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.log.retain-seconds</name>
      <value>36000</value>
  </property>
  <property>
      <name>yarn.nodemanager.delete.debug-delay-sec</name>
      <value>3600</value>
  </property>
  <property>
      <name>yarn.log.server.url</name>
      <value>http://{masterip}:19888/jobhistory/logs/</value>
  </property>

  <property>
      <name>yarn.nodemanager.log-dirs</name>
      <value>/home/{user}/hadoop-2.7.0/logs/userlogs</value>
  </property>
  <property>
      <name>yarn.nodemanager.local-dirs</name>
      <value>/data/1/{user}/yarn/local
      </value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services.spark_shuffle.class</name>
      <value>org.apache.spark.network.yarn.YarnShuffleService</value>
  </property>
</configuration>
'''


with open('hadoop-2.7.0/etc/hadoop/yarn-site.xml','w') as f:
    f.writelines(yarn_site)
    
for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/yarn-site.xml $l:~/hadoop-2.7.0/etc/hadoop/yarn-site.xml >/dev/null 2>&1


## slave config

In [ ]:
with open('hadoop-2.7.0/etc/hadoop/slaves','w') as f:
    f.writelines("\n".join(clients))

for l in clients:
    !scp hadoop-2.7.0/etc/hadoop/slaves $l:~/hadoop-2.7.0/etc/hadoop/ >/dev/null 2>&1


In [ ]:
for l in hclients:
    !scp ./spark-3.0.0-bin-hadoop2.7/yarn/spark-3.0.0-yarn-shuffle.jar $l:~/hadoop-2.7.0/share/hadoop/common/lib/

# start dfs, yarn

In [ ]:
!hadoop-2.7.0/bin/hadoop namenode -format > /dev/null 2>&1

In [ ]:
!hadoop-2.7.0/bin/hadoop datanode -format > /dev/null 2>&1

In [ ]:
for l in clients:
    !ssh $l "ps aux | grep java"

In [ ]:
!hadoop-2.7.0/sbin/start-dfs.sh >/dev/null 2>&1

In [ ]:
!hadoop-2.7.0/sbin/start-yarn.sh >/dev/null 2>&1

In [ ]:
!hadoop-2.7.0/sbin/stop-all.sh >/dev/null 2>&1
!hadoop-2.7.0/sbin/start-all.sh >/dev/null 2>&1

# create dir

In [ ]:
!hadoop fs -mkdir -p /tmp/sparkEventLog

# download data

In [ ]:
client1=clients[0]
#!ssh !client1 "mkdir /data/1/mortgage; cd /data/1/mortgage; wget http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/mortgage_2000-2016.tgz"
!ssh $client1 "cd /data/1/spark/mortgage; tar zxvf mortgage_2000-2016.tgz"

In [ ]:
!ssh $client1 "/home/spark/hadoop-2.7.0/bin/hadoop fs -copyFromLocal /data/1/spark/mortgage /"

In [ ]:
for l in clients:
     !ssh root@$l "echo 3 > /proc/sys/vm/drop_caches"

In [ ]:
!cat /etc/hosts